# Notebook 00 — Open-Source Data Enrichment for Cell-Site FMV

This notebook demonstrates **how to fetch** open-source data that support the [Cell-Site FMV Data Dictionary](../docs/DATA_DICTIONARY_OPEN_SOURCES.md). Each section uses the documented API or bulk download.

**Docs:**
- **Sources & APIs:** [DATA_DICTIONARY_OPEN_SOURCES.md](../docs/DATA_DICTIONARY_OPEN_SOURCES.md)
- **Data locations, table joins, unified dataset:** [DATA_LOCATIONS_AND_UNIFIED_DATASET.md](../docs/DATA_LOCATIONS_AND_UNIFIED_DATASET.md)

**API keys:** Copy `.env.example` to `.env` in the project root and fill in keys (see [Data locations doc](../docs/DATA_LOCATIONS_AND_UNIFIED_DATASET.md)).

---
## 1. Setup and configuration

In [ ]:
import os
import json
import zipfile
import io
import requests
import pandas as pd

# Load .env from project root (one level up from notebooks/)
_cwd = os.getcwd(); _project_root = _cwd if os.path.isdir(os.path.join(_cwd, "data")) else os.path.dirname(_cwd)
_env_path = os.path.join(_project_root, ".env")
if os.path.isfile(_env_path):
    try:
        from dotenv import load_dotenv
        load_dotenv(_env_path)
    except ImportError:
        pass  # pip install python-dotenv to load .env

# API keys (set in .env or environment)
CENSUS_API_KEY = os.environ.get("CENSUS_API_KEY", "")
FRED_API_KEY = os.environ.get("FRED_API_KEY", "")
BLS_API_KEY = os.environ.get("BLS_API_KEY", "")
OPENCELLID_API_KEY = os.environ.get("OPENCELLID_API_KEY", "")

# Data paths — see docs/DATA_LOCATIONS_AND_UNIFIED_DATASET.md
DATA_ROOT = os.path.join(_project_root, "data")
RAW_DIR = os.path.join(DATA_ROOT, "raw")
PROCESSED_DIR = os.path.join(DATA_ROOT, "processed")
UNIFIED_DIR = os.path.join(DATA_ROOT, "unified")
UNIFIED_FILE = os.path.join(UNIFIED_DIR, "sites_enriched.parquet")
for d in (RAW_DIR, PROCESSED_DIR, UNIFIED_DIR):
    os.makedirs(d, exist_ok=True)

print("Project root:", _project_root)
print(".env loaded:", os.path.isfile(_env_path))
print("Census key:", "set" if CENSUS_API_KEY else "not set")
print("FRED key:", "set" if FRED_API_KEY else "not set")
print("OpenCelliD key:", "set" if OPENCELLID_API_KEY else "not set")
print("Unified output:", UNIFIED_FILE)

---
## 2. Census ACS — Tract demographics (X_market)

In [ ]:
# Census ACS 5-year via REST API
# Variables: B01003_001E=pop, B19013_001E=median income, B25077_001E=median home value, B25064_001E=median rent
year = "2022"
state = "36"  # NY
base = f"https://api.census.gov/data/{year}/acs/acs5"
params = {
    "get": "B01003_001E,B19013_001E,B25077_001E,B25064_001E,NAME",
    "for": "tract:*",
    "in": f"state:{state}",
}
if CENSUS_API_KEY:
    params["key"] = CENSUS_API_KEY

r = requests.get(base, params=params)
r.raise_for_status()
data = r.json()
cols = data[0]
rows = data[1:]
df_acs = pd.DataFrame(rows, columns=cols)
df_acs = df_acs.astype({"B01003_001E": float, "B19013_001E": float, "B25077_001E": float, "B25064_001E": float}, errors="ignore")
df_acs.rename(columns={"B01003_001E": "population", "B19013_001E": "median_income", "B25077_001E": "median_home_value", "B25064_001E": "median_rent"}, inplace=True)
print(df_acs.head(10))

---
## 3. Census Geocoder — Address / coordinates → FIPS tract

In [ ]:
# Geographies by coordinates (no key required)
lon, lat = -73.99, 40.71  # example: NYC
url = "https://geocoding.geo.census.gov/geocoder/geographies"
params = {"x": lon, "y": lat, "benchmark": "Public_AR_Current", "vintage": "Current", "format": "json"}
r = requests.get(url, params=params)
r.raise_for_status()
geo = r.json()
if geo.get("result", {}).get("geographies", {}).get("2020 Census Tracts", []):
    tract = geo["result"]["geographies"]["2020 Census Tracts"][0]
    print("Tract FIPS:", tract.get("GEOID"), "|", tract.get("NAME"))
else:
    print(geo)

---
## 4. FRED — Macro indices (CPI, rates)

In [ ]:
# FRED: requires fredapi (pip install fredapi) and FRED_API_KEY
try:
    from fredapi import Fred
    fred = Fred(api_key=FRED_API_KEY or None)
    cpi = fred.get_series("CPIAUCSL", observation_start="2020-01-01")
    print("CPI (last 3):")
    print(cpi.tail(3))
    if FRED_API_KEY:
        d10 = fred.get_series("DGS10", observation_start="2024-01-01")
        print("10Y Treasury (last 3):")
        print(d10.tail(3))
except ImportError:
    print("Install: pip install fredapi. Set FRED_API_KEY for full access.")
except Exception as e:
    print("FRED error (key may be required):", e)

---
## 5. BLS LAUS — County unemployment

In [ ]:
# BLS Public Data API v2 — LAUS county unemployment rate
# Series ID: LAUCN + state FIPS (2) + county FIPS (3) + 0000000 + 003 (unemployment rate)
# Example: LAUCN360610000000003 = Kings County (Brooklyn), NY
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
payload = {
    "seriesid": ["LAUCN360610000000003"],
    "startyear": "2022",
    "endyear": "2024",
}
if BLS_API_KEY:
    payload["registrationkey"] = BLS_API_KEY

r = requests.post(url, json=payload)
r.raise_for_status()
bls = r.json()
if bls.get("status") == "REQUEST_SUCCEEDED":
    for s in bls.get("Results", {}).get("series", []):
        for d in s.get("data", [])[:5]:
            print(d.get("periodName"), d.get("year"), "|", d.get("value"), "%")
else:
    print(bls)

---
## 6. OpenCelliD — Cells in area (tech mix, tower count)

In [ ]:
# OpenCelliD: cells in bounding box. Requires API key (free reg at opencellid.org).
# bb = min_lat,min_lon,max_lat,max_lon
min_lat, min_lon = 40.70, -74.02
max_lat, max_lon = 40.72, -73.98
bb = f"{min_lat},{min_lon},{max_lat},{max_lon}"
url = "https://opencellid.org/cell/getInArea"
params = {"key": OPENCELLID_API_KEY, "bb": bb, "format": "json"}

if not OPENCELLID_API_KEY:
    print("Set OPENCELLID_API_KEY to run. Free reg: https://opencellid.org")
else:
    r = requests.get(url, params=params)
    if r.status_code == 200:
        data = r.json()
        cells = data if isinstance(data, list) else data.get("cells", [])
        print(f"Cells in bbox: {len(cells)}")
        if cells:
            radios = pd.Series([c.get("radio", "") for c in cells]).value_counts()
            print("Radio (tech) distribution:")
            print(radios.head())
    else:
        print(r.status_code, r.text[:200])

---
## 7. FCC ASR — Bulk tower registration download

In [ ]:
# FCC ASR: weekly full registration file (no API key). Pipe-delimited inside zip.
url = "https://data.fcc.gov/download/pub/uls/complete/r_tower.zip"
r = requests.get(url, stream=True)
r.raise_for_status()
z = zipfile.ZipFile(io.BytesIO(r.content))
names = z.namelist()
print("Files in r_tower.zip:", names)
# Parse EN.dat (registration records) — see FCC ASR field definitions for column layout
for name in names:
    if name.endswith(".dat") or name.endswith(".txt"):
        with z.open(name) as f:
            head = f.read(2000).decode("utf-8", errors="ignore")
        print(f"\n--- {name} (first 1500 chars) ---\n", head[:1500])
        break

---
## 8. USGS 3DEP — Point elevation

In [ ]:
# USGS 3DEP via py3dep (pip install py3dep). Returns elevation in meters.
try:
    import py3dep
    coords = [(-73.99, 40.71)]
    elev = py3dep.elevation_bycoords(coords)
    print("Elevation (m) at (40.71, -73.99):", elev)
except ImportError:
    print("Install: pip install py3dep")
except Exception as e:
    print("py3dep error:", e)

---
## 9. FEMA NFHL — Flood zone query (ArcGIS REST)

In [ ]:
# FEMA National Flood Hazard Layer — query by envelope (no key)
base = "https://hazards.fema.gov/arcgis/rest/services/public/NFHL/MapServer/28/query"  # Flood Hazard Zones
xmin, ymin, xmax, ymax = -74.02, 40.70, -73.98, 40.72
params = {
    "where": "1=1",
    "geometry": f"{xmin},{ymin},{xmax},{ymax}",
    "geometryType": "esriGeometryEnvelope",
    "inSR": "4326",
    "outFields": "*",
    "returnGeometry": "false",
    "f": "json",
}
r = requests.get(base, params=params)
r.raise_for_status()
j = r.json()
if "features" in j:
    print(f"Flood zone features in envelope: {len(j['features'])}")
    for f in j["features"][:3]:
        print(f.get("attributes", {}))
else:
    print(j.get("error", j))

---
## 10. HUD SAFMR — Small Area Fair Market Rent (ArcGIS REST)

In [ ]:
# HUD SAFMR Feature Server — query by state/ZIP. Layer 1 = SAFMR_table
base = "https://services.arcgis.com/VTyQ9soqVukalItT/arcgis/rest/services/HUD_PDR_Small_Area_Fair_Market_Rents/FeatureServer/1/query"
params = {
    "where": "STATE = '36'",  # NY
    "outFields": "ZCTA,BR0,BR1,BR2,BR3,BR4",
    "returnGeometry": "false",
    "f": "json",
    "resultRecordCount": 5,
}
r = requests.get(base, params=params)
r.raise_for_status()
j = r.json()
if "features" in j:
    print("SAFMR sample (NY):")
    for f in j["features"]:
        print(f.get("attributes"))
else:
    print(j.get("error", j))

---
## Summary

| Source | Data | Refresh | API / method |
|--------|------|--------|---------------|
| Census ACS | Tract demographics | Annual (5-yr) | REST GET, optional key |
| Census Geocoder | FIPS/tract from coords | TIGER vintage | REST GET, no key |
| FRED | CPI, rates, macro | Monthly/daily | fredapi, key required |
| BLS | County unemployment | Monthly | REST POST, optional key |
| OpenCelliD | Cells in area | Community DB | REST GET, key required (1000/day free) |
| FCC ASR | Tower registrations | Weekly bulk | HTTP zip download, no key |
| USGS 3DEP | Elevation | Periodic | py3dep, no key |
| FEMA NFHL | Flood zones | Continuous | ArcGIS REST, no key |
| HUD SAFMR | SAFMR by ZCTA | Annual | ArcGIS REST, no key |

See **`docs/DATA_DICTIONARY_OPEN_SOURCES.md`** for full attribute mapping and additional sources (FHFA HPI, WRLURI, NLCD, TIGER).